# Hand-off waste calculation

## Pre-processing: grouping traces by case ID

In [65]:
import pandas as pd
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer

# log = xes_importer.apply('data/PurchasingExample.xes')
log = xes_importer.apply('data/PurchasingExampleModified.xes')
event_log = log_converter.apply(log, variant=log_converter.Variants.TO_DATA_FRAME)
event_log_by_case = event_log.groupby(by='case:concept:name')

parsing log, completed traces ::   0%|          | 0/608 [00:00<?, ?it/s]

Case IDs:

In [64]:
event_log_by_case.groups.keys()

dict_keys(['1', '10', '100', '1004', '1008', '1009', '101', '1011', '1015', '1018', '102', '1020', '1021', '1022', '1023', '1025', '103', '1031', '1033', '1035', '104', '1041', '1042', '1044', '1047', '1048', '1052', '1057', '106', '1060', '1063', '1065', '1066', '1068', '107', '1079', '1080', '1085', '1088', '109', '1090', '1094', '11', '110', '1104', '111', '1111', '1114', '1116', '1118', '112', '1123', '1128', '1141', '1144', '1146', '115', '1151', '1153', '1158', '1160', '1161', '117', '1177', '1178', '1179', '118', '1182', '1184', '1187', '119', '1191', '1192', '1197', '1198', '12', '120', '1212', '1214', '1215', '122', '1220', '1225', '1227', '123', '124', '1242', '125', '1255', '1259', '127', '128', '1282', '1284', '1287', '129', '1290', '1294', '1298', '1299', '13', '1300', '1305', '1308', '1309', '1310', '132', '1328', '133', '1337', '1338', '134', '1347', '135', '1353', '1358', '136', '1360', '1361', '1362', '1366', '1367', '1368', '137', '1375', '1377', '1378', '138', '1380'

Case 10

In [68]:
case = event_log_by_case.get_group('10')
case = case.sort_values(by='time:timestamp')
case[['Activity', 'Resource', 'time:timestamp', 'lifecycle:transition']]

,concept:name,lifecycle:transition,org:resource,time:timestamp,Activity,Resource,case:concept:name,case:variant,case:variant-index,case:creator
344,Create Purchase Requisition,start,Esmana Liubiata,2011-01-02 02:52:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
345,Create Purchase Requisition,complete,Esmana Liubiata,2011-01-02 03:43:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
346,Create Request for Quotation,start,Immanuel Karagianni,2011-01-02 10:25:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco
347,Create Request for Quotation,complete,Immanuel Karagianni,2011-01-02 10:32:00-05:00,Create Request for Quotation,Immanuel Karagianni,10,Variant 9,9,Fluxicon Disco
348,Create Purchase Requisition,start,Esmana Liubiata,2011-01-02 10:40:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
349,Create Purchase Requisition,complete,Esmana Liubiata,2011-01-02 11:40:00-05:00,Create Purchase Requisition,Esmana Liubiata,10,Variant 9,9,Fluxicon Disco
350,Analyze Request for Quotation,start,Francois de Perrier,2011-01-02 15:32:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco
351,Analyze Request for Quotation,complete,Francois de Perrier,2011-01-02 15:59:00-05:00,Analyze Request for Quotation,Francois de Perrier,10,Variant 9,9,Fluxicon Disco
352,Send Request for Quotation to Supplier,start,Karel de Groot,2011-01-03 12:50:00-05:00,Send Request for Quotation to Supplier,Karel de Groot,10,Variant 9,9,Fluxicon Disco
353,Send Request for Quotation to Supplier,complete,Karel de Groot,2011-01-03 13:07:00-05:00,Send Request for Quotation to Supplier,Karel de Groot,10,Variant 9,9,Fluxicon Disco


## Handoff Identification (lifecycle log)

In [70]:
resource_changed = case['Resource'] != case.shift(-1)['Resource']
activity_changed = case['Activity'] != case.shift(-1)['Activity']

# because of NaN at the end of the shifted dataframe, we always have True
resource_changed.iloc[-1] = False
activity_changed.iloc[-1] = False

# both conditions must be satisfied
handoff_occurred = resource_changed & activity_changed

case['handoff_occurred'] = handoff_occurred
case[['case:concept:name', 'lifecycle:transition', 'Activity', 'Resource', 'handoff_occurred']]

# TODO: sequential events
#   - identical dates
#   - parallel gateways (enabled timestamp)
#   - modify event log to create a parallel or overlapping activity by time

,case:concept:name,lifecycle:transition,Activity,Resource,handoff_occurred
344,10,start,Create Purchase Requisition,Esmana Liubiata,False
345,10,complete,Create Purchase Requisition,Esmana Liubiata,True
346,10,start,Create Request for Quotation,Immanuel Karagianni,False
347,10,complete,Create Request for Quotation,Immanuel Karagianni,True
348,10,start,Create Purchase Requisition,Esmana Liubiata,False
349,10,complete,Create Purchase Requisition,Esmana Liubiata,True
350,10,start,Analyze Request for Quotation,Francois de Perrier,False
351,10,complete,Analyze Request for Quotation,Francois de Perrier,True
352,10,start,Send Request for Quotation to Supplier,Karel de Groot,False
353,10,complete,Send Request for Quotation to Supplier,Karel de Groot,False


## Handoff Frequency

In [139]:
case.loc[case['handoff_occurred'] == True, 'handoff_frequency'] = 1
case[['case:concept:name', 'Activity', 'Resource', 'handoff_occurred', 'handoff_frequency']]

,case:concept:name,Activity,Resource,handoff_occurred,handoff_frequency
344,10,Create Purchase Requisition,Esmana Liubiata,False,NaN
345,10,Create Purchase Requisition,Esmana Liubiata,True,1.0
346,10,Create Request for Quotation,Immanuel Karagianni,False,NaN
347,10,Create Request for Quotation,Immanuel Karagianni,True,1.0
348,10,Create Purchase Requisition,Esmana Liubiata,False,NaN
349,10,Create Purchase Requisition,Esmana Liubiata,True,1.0
350,10,Analyze Request for Quotation,Francois de Perrier,False,NaN
351,10,Analyze Request for Quotation,Francois de Perrier,True,1.0
352,10,Send Request for Quotation to Supplier,Karel de Groot,False,NaN
353,10,Send Request for Quotation to Supplier,Karel de Groot,False,NaN


## Handoff Duration

In [140]:
handoff_start = case.loc[handoff_occurred, 'time:timestamp']
handoff_end = case.loc[handoff_occurred.shift(1, fill_value=False), 'time:timestamp']
case.loc[handoff_occurred, 'handoff_duration'] = handoff_end.values - handoff_start.values
case[['case:concept:name', 'Activity', 'Resource', 'handoff_occurred', 'handoff_frequency', 'handoff_duration']]

,case:concept:name,Activity,Resource,handoff_occurred,handoff_frequency,handoff_duration
344,10,Create Purchase Requisition,Esmana Liubiata,False,NaN,NaT
345,10,Create Purchase Requisition,Esmana Liubiata,True,1.0,0 days 06:42:00
346,10,Create Request for Quotation,Immanuel Karagianni,False,NaN,NaT
347,10,Create Request for Quotation,Immanuel Karagianni,True,1.0,0 days 00:08:00
348,10,Create Purchase Requisition,Esmana Liubiata,False,NaN,NaT
349,10,Create Purchase Requisition,Esmana Liubiata,True,1.0,0 days 03:52:00
350,10,Analyze Request for Quotation,Francois de Perrier,False,NaN,NaT
351,10,Analyze Request for Quotation,Francois de Perrier,True,1.0,0 days 20:51:00
352,10,Send Request for Quotation to Supplier,Karel de Groot,False,NaN,NaT
353,10,Send Request for Quotation to Supplier,Karel de Groot,False,NaN,NaT


## Handoff Overall Score

In [141]:
case['handoff_score'] = case['handoff_duration'] * case['handoff_frequency']
case[['case:concept:name', 'Activity', 'Resource', 'handoff_occurred', 'handoff_frequency', 'handoff_duration', 'handoff_score']]

,case:concept:name,Activity,Resource,handoff_occurred,handoff_frequency,handoff_duration,handoff_score
344,10,Create Purchase Requisition,Esmana Liubiata,False,NaN,NaT,NaT
345,10,Create Purchase Requisition,Esmana Liubiata,True,1.0,0 days 06:42:00,0 days 06:42:00
346,10,Create Request for Quotation,Immanuel Karagianni,False,NaN,NaT,NaT
347,10,Create Request for Quotation,Immanuel Karagianni,True,1.0,0 days 00:08:00,0 days 00:08:00
348,10,Create Purchase Requisition,Esmana Liubiata,False,NaN,NaT,NaT
349,10,Create Purchase Requisition,Esmana Liubiata,True,1.0,0 days 03:52:00,0 days 03:52:00
350,10,Analyze Request for Quotation,Francois de Perrier,False,NaN,NaT,NaT
351,10,Analyze Request for Quotation,Francois de Perrier,True,1.0,0 days 20:51:00,0 days 20:51:00
352,10,Send Request for Quotation to Supplier,Karel de Groot,False,NaN,NaT,NaT
353,10,Send Request for Quotation to Supplier,Karel de Groot,False,NaN,NaT,NaT


## Handoff Reporting

In [143]:
for (activity, group) in case[case['handoff_score'].notna()].groupby(by='Activity'):
    score_per_activity = group['handoff_score'].sum()
    print(f'{activity}: \t{score_per_activity}')

Analyze Quotation Comparison Map: 	0 days 00:00:00
Analyze Request for Quotation: 	0 days 20:51:00
Approve Purchase Order for payment: 	0 days 21:27:00
Authorize Supplier's Invoice payment: 	0 days 00:20:00
Choose best option: 	0 days 01:48:00
Confirm Purchase Order: 	0 days 08:50:00
Create Purchase Order: 	0 days 16:57:00
Create Purchase Requisition: 	0 days 10:34:00
Create Quotation comparison Map: 	0 days 07:16:00
Create Request for Quotation: 	0 days 00:08:00
Deliver Goods Services: 	0 days 07:35:00
Release Purchase Order: 	1 days 18:00:00
Send Invoice: 	0 days 14:22:00


In [ ]:
# TODO: CTE, processing time / full time

# TODO: handoff types, can we discover resources, calendars (identify human, system, internal or external resource)

# TODO: separate dataframe for metrics

# Concurrent activities

What's the definition of concurrent activities? If a completion timestamp of one event is equal to a start timestamp of another, should it be concurrent (like it's in https://pm4py.fit.fraunhofer.de/documentation#item-10-21) or sequential?

## Pre-processing: converting lifecycle events (has 1 timestamp and lifecycle attribute) to interval events (has 2 timestamps, start and end)

In [117]:
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.util import interval_lifecycle

log = xes_importer.apply('data/PurchasingExampleModified.xes')
log_interval = interval_lifecycle.to_interval(log)
# log_interval_enriched = interval_lifecycle.assign_lead_cycle_time(log_interval)
event_log_interval = log_converter.apply(log_interval, variant=log_converter.Variants.TO_DATA_FRAME)
# event_log_interval['start_timestamp'] = event_log_interval['start_timestamp'] + pd.Timedelta('1us')
event_log_interval

parsing log, completed traces ::   0%|          | 0/608 [00:00<?, ?it/s]

,concept:name,org:resource,Activity,Resource,@@startevent_concept:name,@@startevent_org:resource,@@startevent_Activity,@@startevent_Resource,start_timestamp,time:timestamp,@@duration,case:concept:name,case:variant,case:variant-index,case:creator
0,Create Purchase Requisition,Kim Passa,Create Purchase Requisition,Kim Passa,Create Purchase Requisition,Kim Passa,Create Purchase Requisition,Kim Passa,2011-01-01 00:00:00-05:00,2011-01-01 00:37:00-05:00,2220.0,1,Variant 2,2,Fluxicon Disco
1,Create Request for Quotation,Kim Passa,Create Request for Quotation,Kim Passa,Create Request for Quotation,Kim Passa,Create Request for Quotation,Kim Passa,2011-01-01 05:37:00-05:00,2011-01-01 05:45:00-05:00,480.0,1,Variant 2,2,Fluxicon Disco
2,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,Karel de Groot,2011-01-01 06:41:00-05:00,2011-01-01 06:55:00-05:00,840.0,1,Variant 2,2,Fluxicon Disco
3,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,2011-01-01 11:43:00-05:00,2011-01-01 12:09:00-05:00,1560.0,1,Variant 2,2,Fluxicon Disco
4,Create Quotation comparison Map,Magdalena Predutta,Create Quotation comparison Map,Magdalena Predutta,Create Quotation comparison Map,Magdalena Predutta,Create Quotation comparison Map,Magdalena Predutta,2011-01-01 12:32:00-05:00,2011-01-01 16:03:00-05:00,12660.0,1,Variant 2,2,Fluxicon Disco
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9115,Analyze Purchase Requisition,Heinz Gutschmidt,Analyze Purchase Requisition,Heinz Gutschmidt,Analyze Purchase Requisition,Heinz Gutschmidt,Analyze Purchase Requisition,Heinz Gutschmidt,2011-10-12 18:58:00-05:00,2011-10-12 19:02:00-05:00,240.0,1941,Variant 4,4,Fluxicon Disco
9116,Create Request for Quotation,Francis Odell,Create Request for Quotation,Francis Odell,Create Request for Quotation,Francis Odell,Create Request for Quotation,Francis Odell,2011-10-12 19:06:00-05:00,2011-10-12 19:09:00-05:00,180.0,1941,Variant 4,4,Fluxicon Disco
9117,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,Karel de Groot,Analyze Request for Quotation,Karel de Groot,2011-10-14 14:05:00-05:00,2011-10-14 14:18:00-05:00,780.0,1941,Variant 4,4,Fluxicon Disco
9118,Create Purchase Requisition,Tesca Lobes,Create Purchase Requisition,Tesca Lobes,Create Purchase Requisition,Tesca Lobes,Create Purchase Requisition,Tesca Lobes,2011-10-13 06:27:00-05:00,2011-10-13 07:21:00-05:00,3240.0,1949,Variant 3,3,Fluxicon Disco


## Concurrent activities and its frequencies

In [104]:
from pm4py.statistics.concurrent_activities.pandas import get as concurrent_activities_get

params = {concurrent_activities_get.Parameters.TIMESTAMP_KEY: "time:timestamp", concurrent_activities_get.Parameters.START_TIMESTAMP_KEY: "start_timestamp"}
concurrent_activities = concurrent_activities_get.apply(event_log_interval, parameters=params)
concurrent_activities

{('Analyze Quotation Comparison Map', 'Choose best option'): 413,
 ("Authorize Supplier's Invoice payment", "Release Supplier's Invoice"): 297,
 ("Authorize Supplier's Invoice payment", 'Settle Dispute With Supplier'): 70}

Activities below are considered to be concurrent even though they look like sequential:

In [105]:
event_log_interval[event_log_interval.Activity == 'Authorize Supplier\'s Invoice payment'].head(2)

,concept:name,org:resource,Activity,Resource,@@startevent_concept:name,@@startevent_org:resource,@@startevent_Activity,@@startevent_Resource,start_timestamp,time:timestamp,@@duration,case:concept:name,case:variant,case:variant-index,case:creator
15,Authorize Supplier's Invoice payment,Karalda Nimwada,Authorize Supplier's Invoice payment,Karalda Nimwada,Authorize Supplier's Invoice payment,Karalda Nimwada,Authorize Supplier's Invoice payment,Karalda Nimwada,2011-01-04 15:13:00-05:00,2011-01-04 15:13:00-05:00,0.0,1,Variant 2,2,Fluxicon Disco
37,Authorize Supplier's Invoice payment,Pedro Alvares,Authorize Supplier's Invoice payment,Pedro Alvares,Authorize Supplier's Invoice payment,Pedro Alvares,Authorize Supplier's Invoice payment,Pedro Alvares,2011-01-06 05:58:00-05:00,2011-01-06 05:58:00-05:00,0.0,2,Variant 20,20,Fluxicon Disco


In [106]:
event_log_interval[event_log_interval.Activity == 'Release Supplier\'s Invoice'].head(1)

,concept:name,org:resource,Activity,Resource,@@startevent_concept:name,@@startevent_org:resource,@@startevent_Activity,@@startevent_Resource,start_timestamp,time:timestamp,@@duration,case:concept:name,case:variant,case:variant-index,case:creator
14,Release Supplier's Invoice,Karalda Nimwada,Release Supplier's Invoice,Karalda Nimwada,Release Supplier's Invoice,Karalda Nimwada,Release Supplier's Invoice,Karalda Nimwada,2011-01-04 15:08:00-05:00,2011-01-04 15:13:00-05:00,300.0,1,Variant 2,2,Fluxicon Disco


In [108]:
event_log_interval[event_log_interval.Activity == 'Settle Dispute With Supplier'].head(1)

,concept:name,org:resource,Activity,Resource,@@startevent_concept:name,@@startevent_org:resource,@@startevent_Activity,@@startevent_Resource,start_timestamp,time:timestamp,@@duration,case:concept:name,case:variant,case:variant-index,case:creator
36,Settle Dispute With Supplier,Karalda Nimwada,Settle Dispute With Supplier,Karalda Nimwada,Settle Dispute With Supplier,Karalda Nimwada,Settle Dispute With Supplier,Karalda Nimwada,2011-01-06 05:38:00-05:00,2011-01-06 05:58:00-05:00,1200.0,2,Variant 20,20,Fluxicon Disco


# Hand-off waste calculation (interval log)

In [109]:
import pandas as pd

# Adjusting time slightly to avoid unnecessary concurrent events
# event_log_interval['start_timestamp'] = event_log_interval['start_timestamp'] + pd.Timedelta('1us')

event_log_interval_by_case = event_log_interval.groupby(by='case:concept:name')
case = event_log_interval_by_case.get_group('10').sort_values(by='start_timestamp')

Case 10

In [110]:
case

,concept:name,org:resource,Activity,Resource,@@startevent_concept:name,@@startevent_org:resource,@@startevent_Activity,@@startevent_Resource,start_timestamp,time:timestamp,@@duration,case:concept:name,case:variant,case:variant-index,case:creator
172,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,2011-01-02 02:52:00-05:00,2011-01-02 03:43:00-05:00,3060.0,10,Variant 9,9,Fluxicon Disco
173,Create Request for Quotation,Immanuel Karagianni,Create Request for Quotation,Immanuel Karagianni,Create Request for Quotation,Immanuel Karagianni,Create Request for Quotation,Immanuel Karagianni,2011-01-02 10:25:00-05:00,2011-01-02 10:32:00-05:00,420.0,10,Variant 9,9,Fluxicon Disco
174,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,2011-01-02 10:40:00-05:00,2011-01-02 11:40:00-05:00,3600.0,10,Variant 9,9,Fluxicon Disco
175,Analyze Request for Quotation,Francois de Perrier,Analyze Request for Quotation,Francois de Perrier,Analyze Request for Quotation,Francois de Perrier,Analyze Request for Quotation,Francois de Perrier,2011-01-02 15:32:00-05:00,2011-01-02 15:59:00-05:00,1620.0,10,Variant 9,9,Fluxicon Disco
176,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,2011-01-03 12:50:00-05:00,2011-01-03 13:07:00-05:00,1020.0,10,Variant 9,9,Fluxicon Disco
177,Create Quotation comparison Map,Karel de Groot,Create Quotation comparison Map,Karel de Groot,Create Quotation comparison Map,Karel de Groot,Create Quotation comparison Map,Karel de Groot,2011-01-03 13:31:00-05:00,2011-01-03 14:28:00-05:00,3420.0,10,Variant 9,9,Fluxicon Disco
178,Analyze Quotation Comparison Map,Anna Kaufmann,Analyze Quotation Comparison Map,Anna Kaufmann,Analyze Quotation Comparison Map,Anna Kaufmann,Analyze Quotation Comparison Map,Anna Kaufmann,2011-01-03 21:44:00-05:00,2011-01-03 22:05:00-05:00,1260.0,10,Variant 9,9,Fluxicon Disco
179,Choose best option,Nico Ojenbeer,Choose best option,Nico Ojenbeer,Choose best option,Nico Ojenbeer,Choose best option,Nico Ojenbeer,2011-01-03 22:05:00-05:00,2011-01-03 22:05:00-05:00,0.0,10,Variant 9,9,Fluxicon Disco
180,Settle Conditions With Supplier,Karel de Groot,Settle Conditions With Supplier,Karel de Groot,Settle Conditions With Supplier,Karel de Groot,Settle Conditions With Supplier,Karel de Groot,2011-01-03 23:53:00-05:00,2011-01-04 04:31:00-05:00,16680.0,10,Variant 9,9,Fluxicon Disco
181,Create Purchase Order,Karel de Groot,Create Purchase Order,Karel de Groot,Create Purchase Order,Karel de Groot,Create Purchase Order,Karel de Groot,2011-01-04 09:44:00-05:00,2011-01-04 09:52:00-05:00,480.0,10,Variant 9,9,Fluxicon Disco


Concurrent activities in Case 10:

In [111]:
params = {concurrent_activities_get.Parameters.TIMESTAMP_KEY: "time:timestamp", concurrent_activities_get.Parameters.START_TIMESTAMP_KEY: "start_timestamp"}
concurrent_activities = concurrent_activities_get.apply(case, parameters=params)
concurrent_activities

{('Analyze Quotation Comparison Map', 'Choose best option'): 1}

For concurrent activities we use the concept of **enabled timestamp** to calculate the timestamp of completion of concurrent activities. From a set of concurrent activities, the enabled time stamp is the one that ends the latest.

We remove concurrent activities from the case and *add only one with the latest completion timestamp*.

In [81]:
case_without_concurrent_activities = case.copy()

for activities_pair in concurrent_activities:
    concurrent = case[(case['Activity'] == activities_pair[0]) | (case['Activity'] == activities_pair[1])]
    case_without_concurrent_activities.drop(concurrent.index, inplace=True)
    if concurrent.iloc[0]['time:timestamp'] >= concurrent.iloc[1]['time:timestamp']:
        print(f"Adding \"{concurrent.iloc[0]['Activity']}\", dropping \"{concurrent.iloc[1]['Activity']}\"")
        case_without_concurrent_activities = case_without_concurrent_activities.append(concurrent.iloc[0])
    else:
        print(f"Adding \"{concurrent.iloc[1]['Activity']}\", dropping \"{concurrent.iloc[0]['Activity']}\"")
        case_without_concurrent_activities = case_without_concurrent_activities.append(concurrent.iloc[1])

case_without_concurrent_activities.sort_values(by='start_timestamp', inplace=True)
case_without_concurrent_activities.reset_index(drop=True, inplace=True)
case_without_concurrent_activities

Adding "Analyze Quotation Comparison Map", dropping "Choose best option"


,concept:name,org:resource,Activity,Resource,@@startevent_concept:name,@@startevent_org:resource,@@startevent_Activity,@@startevent_Resource,start_timestamp,time:timestamp,@@duration,case:concept:name,case:variant,case:variant-index,case:creator
0,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,2011-01-02 02:52:00-05:00,2011-01-02 03:43:00-05:00,3060.0,10,Variant 9,9,Fluxicon Disco
1,Create Request for Quotation,Immanuel Karagianni,Create Request for Quotation,Immanuel Karagianni,Create Request for Quotation,Immanuel Karagianni,Create Request for Quotation,Immanuel Karagianni,2011-01-02 10:25:00-05:00,2011-01-02 10:32:00-05:00,420.0,10,Variant 9,9,Fluxicon Disco
2,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,Create Purchase Requisition,Esmana Liubiata,2011-01-02 10:40:00-05:00,2011-01-02 11:40:00-05:00,3600.0,10,Variant 9,9,Fluxicon Disco
3,Analyze Request for Quotation,Francois de Perrier,Analyze Request for Quotation,Francois de Perrier,Analyze Request for Quotation,Francois de Perrier,Analyze Request for Quotation,Francois de Perrier,2011-01-02 15:32:00-05:00,2011-01-02 15:59:00-05:00,1620.0,10,Variant 9,9,Fluxicon Disco
4,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,Send Request for Quotation to Supplier,Karel de Groot,2011-01-03 12:50:00-05:00,2011-01-03 13:07:00-05:00,1020.0,10,Variant 9,9,Fluxicon Disco
5,Create Quotation comparison Map,Karel de Groot,Create Quotation comparison Map,Karel de Groot,Create Quotation comparison Map,Karel de Groot,Create Quotation comparison Map,Karel de Groot,2011-01-03 13:31:00-05:00,2011-01-03 14:28:00-05:00,3420.0,10,Variant 9,9,Fluxicon Disco
6,Analyze Quotation Comparison Map,Anna Kaufmann,Analyze Quotation Comparison Map,Anna Kaufmann,Analyze Quotation Comparison Map,Anna Kaufmann,Analyze Quotation Comparison Map,Anna Kaufmann,2011-01-03 21:44:00-05:00,2011-01-03 22:05:00-05:00,1260.0,10,Variant 9,9,Fluxicon Disco
7,Settle Conditions With Supplier,Karel de Groot,Settle Conditions With Supplier,Karel de Groot,Settle Conditions With Supplier,Karel de Groot,Settle Conditions With Supplier,Karel de Groot,2011-01-03 23:53:00-05:00,2011-01-04 04:31:00-05:00,16680.0,10,Variant 9,9,Fluxicon Disco
8,Create Purchase Order,Karel de Groot,Create Purchase Order,Karel de Groot,Create Purchase Order,Karel de Groot,Create Purchase Order,Karel de Groot,2011-01-04 09:44:00-05:00,2011-01-04 09:52:00-05:00,480.0,10,Variant 9,9,Fluxicon Disco
9,Confirm Purchase Order,Carmen Finacse,Confirm Purchase Order,Carmen Finacse,Confirm Purchase Order,Carmen Finacse,Confirm Purchase Order,Carmen Finacse,2011-01-05 02:49:00-05:00,2011-01-05 02:58:00-05:00,540.0,10,Variant 9,9,Fluxicon Disco


## Handoff Identification

In [112]:
handoff = pd.DataFrame(columns=['source_activity', 'source_resource', 'destination_activity', 'destination_resource', 'duration'])

case = case_without_concurrent_activities

resource_changed = case['Resource'] != case.shift(-1)['Resource']
activity_changed = case['Activity'] != case.shift(-1)['Activity']

# because of NaN at the end of the shifted dataframe, we always have True
# resource_changed.iloc[-1] = False
# activity_changed.iloc[-1] = False

# both conditions must be satisfied
handoff_occurred = resource_changed & activity_changed
handoff_occurred

0      True
1      True
2      True
3      True
4     False
5      True
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14    False
15    False
16     True
17     True
dtype: bool

The whole case (without concurrent activities):

In [87]:
case[['Activity', 'Resource', 'start_timestamp', 'time:timestamp', '@@duration']]

,Activity,Resource,start_timestamp,time:timestamp,@@duration
0,Create Purchase Requisition,Esmana Liubiata,2011-01-02 02:52:00-05:00,2011-01-02 03:43:00-05:00,3060.0
1,Create Request for Quotation,Immanuel Karagianni,2011-01-02 10:25:00-05:00,2011-01-02 10:32:00-05:00,420.0
2,Create Purchase Requisition,Esmana Liubiata,2011-01-02 10:40:00-05:00,2011-01-02 11:40:00-05:00,3600.0
3,Analyze Request for Quotation,Francois de Perrier,2011-01-02 15:32:00-05:00,2011-01-02 15:59:00-05:00,1620.0
4,Send Request for Quotation to Supplier,Karel de Groot,2011-01-03 12:50:00-05:00,2011-01-03 13:07:00-05:00,1020.0
5,Create Quotation comparison Map,Karel de Groot,2011-01-03 13:31:00-05:00,2011-01-03 14:28:00-05:00,3420.0
6,Analyze Quotation Comparison Map,Anna Kaufmann,2011-01-03 21:44:00-05:00,2011-01-03 22:05:00-05:00,1260.0
7,Settle Conditions With Supplier,Karel de Groot,2011-01-03 23:53:00-05:00,2011-01-04 04:31:00-05:00,16680.0
8,Create Purchase Order,Karel de Groot,2011-01-04 09:44:00-05:00,2011-01-04 09:52:00-05:00,480.0
9,Confirm Purchase Order,Carmen Finacse,2011-01-05 02:49:00-05:00,2011-01-05 02:58:00-05:00,540.0


Handoff identified between the following activities and resources:

In [88]:
case[handoff_occurred][['Activity', 'Resource', 'start_timestamp', 'time:timestamp', '@@duration']]


,Activity,Resource,start_timestamp,time:timestamp,@@duration
0,Create Purchase Requisition,Esmana Liubiata,2011-01-02 02:52:00-05:00,2011-01-02 03:43:00-05:00,3060.0
1,Create Request for Quotation,Immanuel Karagianni,2011-01-02 10:25:00-05:00,2011-01-02 10:32:00-05:00,420.0
2,Create Purchase Requisition,Esmana Liubiata,2011-01-02 10:40:00-05:00,2011-01-02 11:40:00-05:00,3600.0
3,Analyze Request for Quotation,Francois de Perrier,2011-01-02 15:32:00-05:00,2011-01-02 15:59:00-05:00,1620.0
5,Create Quotation comparison Map,Karel de Groot,2011-01-03 13:31:00-05:00,2011-01-03 14:28:00-05:00,3420.0
6,Analyze Quotation Comparison Map,Anna Kaufmann,2011-01-03 21:44:00-05:00,2011-01-03 22:05:00-05:00,1260.0
8,Create Purchase Order,Karel de Groot,2011-01-04 09:44:00-05:00,2011-01-04 09:52:00-05:00,480.0
9,Confirm Purchase Order,Carmen Finacse,2011-01-05 02:49:00-05:00,2011-01-05 02:58:00-05:00,540.0
10,Deliver Goods Services,Kiu Kan,2011-01-05 11:48:00-05:00,2011-01-07 01:19:00-05:00,135060.0
11,Release Purchase Order,Tesca Lobes,2011-01-07 08:54:00-05:00,2011-01-07 08:55:00-05:00,60.0


In [90]:
handoff.loc[:, 'source_activity'] = case[handoff_occurred]['Activity']
handoff.loc[:, 'source_resource'] = case[handoff_occurred]['Resource']
handoff.loc[:, 'destination_activity'] = case[handoff_occurred].shift(-1)['Activity']
handoff.loc[:, 'destination_resource'] = case[handoff_occurred].shift(-1)['Resource']
handoff['duration'] = case[handoff_occurred].shift(-1)['start_timestamp'] - case[handoff_occurred]['time:timestamp']

In [93]:
handoff.drop(handoff.tail(1).index, inplace=True)

Handoff with duration:

In [94]:
handoff


,source_activity,source_resource,destination_activity,destination_resource,duration
0,Create Purchase Requisition,Esmana Liubiata,Create Request for Quotation,Immanuel Karagianni,0 days 06:42:00
1,Create Request for Quotation,Immanuel Karagianni,Create Purchase Requisition,Esmana Liubiata,0 days 00:08:00
2,Create Purchase Requisition,Esmana Liubiata,Analyze Request for Quotation,Francois de Perrier,0 days 03:52:00
3,Analyze Request for Quotation,Francois de Perrier,Create Quotation comparison Map,Karel de Groot,0 days 21:32:00
5,Create Quotation comparison Map,Karel de Groot,Analyze Quotation Comparison Map,Anna Kaufmann,0 days 07:16:00
6,Analyze Quotation Comparison Map,Anna Kaufmann,Create Purchase Order,Karel de Groot,0 days 11:39:00
8,Create Purchase Order,Karel de Groot,Confirm Purchase Order,Carmen Finacse,0 days 16:57:00
9,Confirm Purchase Order,Carmen Finacse,Deliver Goods Services,Kiu Kan,0 days 08:50:00
10,Deliver Goods Services,Kiu Kan,Release Purchase Order,Tesca Lobes,0 days 07:35:00
11,Release Purchase Order,Tesca Lobes,Approve Purchase Order for payment,Karel de Groot,1 days 18:00:00
